In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot

import sys

D:\User\Anaconda3\envs\rl_fx\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

<a id='1.4'></a>
## 2.4. Create Folders

In [6]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [7]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [8]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

ticker_list is a list of stock tickers, in a single stock trading case, the list contains only 1 ticker

In [9]:
from finrl.preprocessing import data, preprocessors
data_df=data.load_ohlc_dataset("chicago_pmi/EURUSD/ohlc/EURUSD_Chicago_Pmi_2018-01-31.csv")

<a id='3'></a>
# Part 4. Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* FinRL uses a class **FeatureEngineer** to preprocess the data
* Add **technical indicators**. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc.


<a id='3.1'></a>

## 4.1 Technical Indicators
* FinRL uses stockstats to calcualte technical indicators such as **Moving Average Convergence Divergence (MACD)**, **Relative Strength Index (RSI)**, **Average Directional Index (ADX)**, **Commodity Channel Index (CCI)** and other various indicators and stats.
* **stockstats**: supplies a wrapper StockDataFrame based on the **pandas.DataFrame** with inline stock statistics/indicators support.



In [10]:
param_map = {
                'sma_9': {'talib_name' : 'SMA', 'time_period' : 9}, #time_period in seoonds
                'ema_9': {'talib_name' : 'EMA', 'time_period' : 9}, #time_period in seoonds
                'sma_21' : {'talib_name' : 'SMA', 'time_period' : 21},
                'ema_21' : {'talib_name' : 'EMA', 'time_period' : 21}
}

<a id='3.2'></a>
## 4.2 Perform Feature Engineering

In [11]:
from finrl.preprocessing import data, preprocessors

data_df = preprocessors.FeatureEngineer(data_df,
                          tech_indicator_params_map = param_map,
                          use_technical_indicator=True,
                          user_defined_feature=False,
                          tech_indicator_list = ["sma_9", "ema_9", "sma_21", "ema_21"]).preprocess_data()

Successfully added technical indicators


For now, let's further preprocess the data to be a simple OHLC format with date

In [12]:
train = data_df['ovr'].iloc[29:]
train.reset_index(inplace=True, col_fill='date')
train.rename(columns={
    "time" : "date"
}, inplace=True)
train

,date,open,high,low,close,volume,tic,sma_9,ema_9,sma_21,ema_21
0,2018-01-31 13:45:29+00:00,1.246495,1.246505,1.246495,1.246505,4.570000,FX,1.246571,1.246571,1.246571,1.246571
1,2018-01-31 13:45:30+00:00,1.246495,1.246560,1.246495,1.246560,7.340000,FX,1.246568,1.246571,1.246568,1.246571
2,2018-01-31 13:45:31+00:00,1.246550,1.246550,1.246520,1.246520,18.480000,FX,1.246563,1.246567,1.246563,1.246567
3,2018-01-31 13:45:32+00:00,1.246510,1.246510,1.246460,1.246475,11.060000,FX,1.246557,1.246561,1.246557,1.246561
4,2018-01-31 13:45:33+00:00,1.246485,1.246485,1.246430,1.246430,13.510000,FX,1.246549,1.246553,1.246549,1.246553
...,...,...,...,...,...,...,...,...,...,...,...
7166,2018-01-31 15:44:55+00:00,1.247040,1.247040,1.247040,1.247040,6.490000,FX,1.247078,1.247057,1.247078,1.247057
7167,2018-01-31 15:44:56+00:00,1.247035,1.247060,1.247035,1.247060,8.860000,FX,1.247079,1.247057,1.247079,1.247057
7168,2018-01-31 15:44:57+00:00,1.247075,1.247080,1.247060,1.247065,28.630001,FX,1.247080,1.247058,1.247080,1.247058
7169,2018-01-31 15:44:58+00:00,1.247060,1.247060,1.247060,1.247060,8.690000,FX,1.247081,1.247058,1.247081,1.247058


<a id='4'></a>
# Part 5. Build Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

<a id='4.1'></a>
## 5.1 Training & Trade data split
* Training: 2009-01-01 to 2018-12-31
* Trade: 2019-01-01 to 2020-09-30

<a id='4.2'></a>
## 5.2 User-defined Environment: a simulation environment class 

<a id='4.3'></a>
## 5.3 Initialize Environment
* **stock dimension**: the number of unique stock tickers we use
* **hmax**: the maximum amount of shares to buy or sell
* **initial amount**: the amount of money we use to trade in the begining
* **transaction cost percentage**: a per share rate for every share trade
* **tech_indicator_list**: a list of technical indicator names (modified from config.py)

In [13]:
## we store the stockstats technical indicator column names in config.py
## check https://github.com/jealous/stockstats for different names
tech_indicator_list = list(param_map.keys())
tech_indicator_list

['sma_9', 'ema_9', 'sma_21', 'ema_21']

In [14]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(tech_indicator_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 7


In [15]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 100000, 
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)
print(e_train_gym.data)

state : {self.state}
date      2018-01-31 13:45:29+00:00
open                        1.24649
high                         1.2465
low                         1.24649
close                        1.2465
volume                         4.57
tic                              FX
sma_9                       1.24657
ema_9                       1.24657
sma_21                      1.24657
ema_21                      1.24657
Name: 0, dtype: object


In [33]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

state : {self.state}
<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [34]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC



### Model 1: A2C

In [35]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

     |
|    portfolio_value    | 9.93e+04 |
|    total_cost         | 791      |
|    total_reward       | -741     |
|    total_reward_pct   | -0.741   |
|    total_trades       | 7153     |
| rollout/              |          |
|    ep_len_mean        | 7.17e+03 |
|    ep_rew_mean        | -0.0686  |
| time/                 |          |
|    fps                | 490      |
|    iterations         | 7200     |
|    time_elapsed       | 73       |
|    total_timesteps    | 36000    |
| train/                |          |
|    entropy_loss       | -2.8     |
|    explained_variance | nan      |
|    learning_rate      | 0.0002   |
|    n_updates          | 7199     |
|    policy_loss        | 1.75e-05 |
|    std                | 3.99     |
|    value_loss         | 1.21e-10 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.17e+03  |
|    ep_rew_mean        | -0.0686   |
| time/                 |  

In [45]:
trained_a2c.env.envs[0].get_episode_rewards()

[-0.059186494011029024,
 -0.06558558034347822,
 -0.07023734626050877,
 -0.07379768521003421,
 -0.07411650664196089,
 -0.0795954912249785]

In [30]:
from stable_baselines3.common.evaluation import evaluate_policy

test = data.load_ohlc_dataset("chicago_pmi/EURUSD/ohlc/EURUSD_Chicago_Pmi_2019-12-30.csv")

test = preprocessors.FeatureEngineer(test,
                          tech_indicator_params_map = param_map,
                          use_technical_indicator=True,
                          user_defined_feature=False,
                          tech_indicator_list = ["sma_9", "ema_9", "sma_21", "ema_21"]).preprocess_data()
test = test['ovr'].iloc[29:]
test.reset_index(inplace=True, col_fill='date')
test.rename(columns={
    "time" : "date"
}, inplace=True)

e_test_gym = StockTradingEnv(df = test, **env_kwargs)
env_test = e_test_gym.get_sb_env()



def wrapper_rl_algo(ent_coef, learning_rate):
    agent = DRLAgent(env = env_train)
    model = agent.get_model(model_name=model_name, model_kwargs = params)
    trained_model = agent.train_model(model=model, 
                                tb_log_name=model_name,
                                total_timesteps=3600*2)
    mean, std = evaluate_policy(trained_model, env_test, 10, True, True)
    print(f"mean: {mean}")
    print(f"std: {std}")
    return mean
    

    


Successfully added technical indicators
state : {self.state}
state : {self.state}


In [31]:
from bayes_opt import BayesianOptimization

pbounds = {'ent_coef': (0.0001, 0.02), 'learning_rate': (0.00001, 0.002)}

optimizer = BayesianOptimization(
    f=wrapper_rl_algo,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [33]:
optimizer.maximize(init_points=2, n_iter=3)

|   iter    |  target   | ent_coef  | learni... |
-------------------------------------------------


TypeError: wrapper_rl_algo() missing 3 required positional arguments: 'model_name', 'env_train', and 'env_test'

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000, "learning_rate": 0.0001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        DDPG_PARAMS = {"batch_size": 64, "buffer_size": 500000+(i-10)*10000, "learning_rate": 0.0001+(j-10)/200000}
        model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)
        trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=30000)
        print('i:',i,'j:',j)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)

In [ ]:
agent = DRLAgent(env = env_train)

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005+(i-10)/1000,
    "learning_rate": 0.0001+(j-10)/200000,
    "batch_size": 128,
}
        
        model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)
        trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)
        print('i:',i,'j:',j)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000, 
              "learning_rate": 0.0003}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000, 
              "learning_rate": 0.0003}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

for i in range(20):
    for j in range(20):
        agent = DRLAgent(env = env_train)
        TD3_PARAMS = {"batch_size": 128, 
              "buffer_size": 1000000+(i-10)*100000, 
              "learning_rate": 0.0003+(j-10)/20000}

        model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)
        trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)
        print('i:',i,'j:',j)

### Model 4: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.00003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

In [ ]:
agent = DRLAgent(env = env_train)

SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.00003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

for i in range(20):
    for j in range(20):
        for k in range(20):
            agent = DRLAgent(env = env_train)
            SAC_PARAMS = {
        "batch_size": 128,
        "buffer_size": 100000+(i-10)*10000,
        "learning_rate": 0.00003+(j-10)/200000,
        "learning_starts": 100,
        "ent_coef": "auto_0.1",
    }
            model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)
            trained_sac = agent.train_model(model=model_sac, 
                                tb_log_name='sac',
                                total_timesteps=30000)
            print('i:',i,'j:',j,'k:',k)

### Trading
* we use the environment class we initialized at 5.3 to create a stock trading environment
* Assume that we have $100,000 initial capital at 2019-01-01. 
* We use the trained model of PPO to trade AAPL.

In [24]:
## make a prediction and get the account value change
trade = data.load_ohlc_dataset("chicago_pmi/EURUSD/ohlc/EURUSD_Chicago_Pmi_2018-02-28.csv")

trade = preprocessors.FeatureEngineer(trade,
                          tech_indicator_params_map = param_map,
                          use_technical_indicator=True,
                          user_defined_feature=False,
                          tech_indicator_list = ["sma_9", "ema_9", "sma_21", "ema_21"]).preprocess_data()
trade = data_df['ovr'].iloc[29:]
trade.reset_index(inplace=True, col_fill='date')
trade.rename(columns={
    "time" : "date"
}, inplace=True)



e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                                           test_data = trade,
                                           test_env = env_trade,
                                           test_obs = obs_trade)

Successfully added technical indicators
state : {self.state}
state : {self.state}
state : {self.state}


<a id='6'></a>
# Part 7: Backtesting Performance
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [26]:
%autoreload

In [27]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = BackTestStats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
annual return:  -0.027387840248382922
sharpe ratio:  -1.7959558934770594
Annual return         -0.000274
Cumulative returns    -0.007764
Annual volatility      0.000153
Sharpe ratio          -1.795956
Calmar ratio          -0.034999
Stability              0.993574
Max drawdown          -0.007824
Omega ratio            0.673116
Sortino ratio         -2.339303
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.899962
Daily value at risk   -0.000020
Alpha                  0.000000
Beta                   1.000000
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to AAPL itself buy-and-hold===========")
%matplotlib inline
BackTestPlot(account_value=df_account_value, 
             baseline_ticker = 'AAPL',
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')

<a id='6.3'></a>
## 7.3 Baseline Stats

In [ ]:
print("==============Get Baseline Stats===========")
baesline_perf_stats=BaselineStats('AAPL')

In [ ]:
print("==============Get Baseline Stats===========")
baesline_perf_stats=BaselineStats('^GSPC')

<a id='6.4'></a>
## 7.4 Compare to Stock Market Index

In [ ]:
print("==============Compare to S&P 500===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
BackTestPlot(df_account_value, baseline_ticker = '^GSPC')